In [1]:
"""
Inventory management simulation code.

Written by Alex Mina in 2024.
"""

'\nInventory management simulation code.\n\nWritten by Alex Mina in 2024.\n'

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import math
import plotly.graph_objects as go
from scipy.stats import norm
from typing import List, Dict, Tuple

Seed for reproducibility, change it if you want different demand or stochastic lead times

In [3]:
seed = 42
np.random.seed(seed)

Generate a time series that will simulate the weekly sales of our products in a two years period

In [4]:
# Define the date range
date_range = pd.date_range(start="2021-01-01", end="2023-12-31", freq='W')

# Define the random number boundaries
lower_bound = 50
upper_bound = 200


# Generate the random numbers
random_numbers = np.random.randint(lower_bound, upper_bound, size=len(date_range))

# Create a time series
time_series = pd.Series(data=random_numbers, index=date_range)

# Create a time series DataFrame
df = pd.DataFrame({'Date': date_range, 'Value': random_numbers})


Plot the time series

In [5]:
# Plot the time series using plotly
fig = px.line(df, x='Date', y='Value', title='Generated product sales from 2021-01-01 to 2023-12-31')
fig.show()

Set the service level values

Note that as you approach a 100% service level, the marginal cost increases rapidly.

In [6]:
# Generate a dictionary of Z-values for given service levels
def generate_service_level_z_values():
    service_levels = [20, 30, 50, 70, 80, 85, 90, 95, 97, 98, 99, 99.5, 99.9]
    z_values = {level: round(norm.ppf(level / 100), 2) for level in service_levels}
    return z_values

# Example usage
z_values_dict = generate_service_level_z_values()


# Plot using plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(z_values_dict.keys()), y=list(z_values_dict.values()), mode='lines+markers', name='Z-values'))

fig.update_layout(title='Z-values for Different Service Levels',
                  xaxis_title='Service Level (%)',
                  yaxis_title='Z-value',
                  xaxis=dict(tickmode='array', tickvals=list(z_values_dict.keys())),
                  yaxis=dict(tickformat=".2f"))

fig.show()

This is how we calculate the safety stock and the reorder point

In [7]:
def calculate_safety_stock(service_level_z: float, std_demand: float, review_period: int, lead_time: int) -> float:
    """
    Calculate the safety stock required to meet the desired service level.
    Formula from Tempelmeier (2011)

    Parameters:
    - service_level_z (float): Z-score for the desired service level.
    - std_demand (float): Standard deviation of demand.
    - review_period (int): Time period between reviews.
    - lead_time (int): Lead time for orders.

    Returns:
    - float: Safety stock level.
    """
    return service_level_z * std_demand * np.sqrt(review_period/2 + lead_time)

def calculate_reorder_point(avg_demand: float, lead_time: int, review_period: int, safety_stock: float) -> float:
    """
    Calculate the reorder point at which a new order should be placed.
    Formula from Tempelmeier (2011)

    Parameters:
    - avg_demand (float): Average demand.
    - lead_time (int): Lead time for orders.
    - review_period (int): Time period between reviews.
    - safety_stock (float): Safety stock level.

    Returns:
    - float: Reorder point.
    """
    return avg_demand * (lead_time + review_period/2) + safety_stock


def update_stock_level(stock: int, current_demand: int) -> int:
    """
    Update the stock level based on current demand.

    Parameters:
    - stock (int): Current stock level.
    - current_demand (int): Demand for the current period.

    Returns:
    - int: Updated stock level.
    """
    return max(stock - current_demand, 0)

def receive_orders(stock: int, orders_backlog: List[Dict], date: pd.Timestamp) -> Tuple[int, List[Dict]]:
    """
    Process received orders and update stock levels.

    Parameters:
    - stock (int): Current stock level.
    - orders_backlog (List[Dict]): List of pending orders.
    - date (pd.Timestamp): Current date.

    Returns:
    - int: Updated stock level.
    - List[Dict]: Updated list of pending orders.
    """
    # Filter orders arriving today and update stock
    orders_to_receive = [order for order in orders_backlog if order['arrival_date'] == date]
    for order in orders_to_receive:
        stock += order['quantity']
    # Remove received orders from backlog
    return stock, [order for order in orders_backlog if order['arrival_date'] != date]


In [8]:
def simulate_inventory(time_series: pd.Series, initial_weeks: int, review_period: int, lead_time: int,
                       service_level_z: float, min_order: int = 0, ignore_pending_orders: bool = False,
                       stochastic_lead_time: bool = False, plot_simulation: bool = False) -> pd.DataFrame:
    """
    Simulate inventory management over a given time series of demand data.

    Parameters:
    - time_series (pd.Series): Time series of demand data.
    - initial_weeks (int): Number of weeks used to calculate initial parameters.
    - review_period (int): Time period between inventory reviews.
    - lead_time (int): Lead time for orders.
    - service_level_z (float): Z-score for the desired service level.
    - min_order (int, optional): Minimum order quantity. Defaults to 0.
    - ignore_pending_orders (bool, optional): Whether to ignore pending orders in calculations. Defaults to False.
    - stochastic_lead_time (bool, optional): Whether to simulate a stochastic or deterministic lead time. Defaults to False.
    - plot_simulation (bool, optional): Whether to plot the simulation results. Defaults to False.

    Returns:
    - pd.DataFrame: DataFrame containing the simulation results.
    """

    # Initial calculations based on the initial period
    initial_period = time_series[:initial_weeks]
    avg_demand = initial_period.mean()
    std_demand = initial_period.std()
    time_series = time_series[initial_weeks:]

    # Initial safety stock and reorder point
    safety_stock = calculate_safety_stock(service_level_z, std_demand, review_period, lead_time)
    reorder_point = calculate_reorder_point(avg_demand, lead_time, review_period, safety_stock)
    stock = reorder_point  # Initial stock level
    orders_backlog = []

    # Historical data lists
    historical_data = {
        'Date': [],
        'Stock': [],
        'ReorderPoint': [],
        'SafetyStock': [],
        'Orders': []
    }
    historical_lead_time = []
    avg_lead_time = lead_time

    for i, date in enumerate(time_series.index):
        current_demand = time_series[date]

        reorder_quantity = 0

        # Update stock level based on current demand
        stock = update_stock_level(stock, current_demand)

        # Process received orders
        stock, orders_backlog = receive_orders(stock, orders_backlog, date)

        # Review inventory and place new orders if necessary
        if i % review_period == 0:
            pending_orders = sum(order['quantity'] for order in orders_backlog) if not ignore_pending_orders else 0
            avg_demand = time_series[:date].mean()
            std_demand = time_series[:date].std()
            safety_stock = calculate_safety_stock(service_level_z, std_demand, review_period, avg_lead_time)
            # this is a simplification
            # different pending orders will be delivered in different days, we should also consider this in the formula
            reorder_point = calculate_reorder_point(avg_demand, avg_lead_time, review_period, safety_stock) - pending_orders

            if stock <= reorder_point:
                demand_over_lead_time = avg_demand * lead_time
                demand_over_review_time = avg_demand * review_period
                target_inventory = demand_over_lead_time + demand_over_review_time + safety_stock
                reorder_quantity = max(0, target_inventory - (stock + pending_orders))

                real_lead_time = lead_time
                if stochastic_lead_time:
                  adjusted_std_dev = lead_time / 2
                  real_lead_time = int(math.ceil(max(1, np.random.normal(lead_time, adjusted_std_dev))))

                historical_lead_time.append(real_lead_time)

                if reorder_quantity >= min_order:
                    orders_backlog.append({'arrival_date': date + pd.Timedelta(weeks=real_lead_time), 'quantity': reorder_quantity})
                else:
                  reorder_quantity = 0

        # Record historical data
        historical_data['Date'].append(date)
        historical_data['Stock'].append(stock)
        historical_data['ReorderPoint'].append(reorder_point)
        historical_data['SafetyStock'].append(safety_stock)
        historical_data['Orders'].append(reorder_quantity)

        # Update average lead time
        if historical_lead_time:
            avg_lead_time = int(round(np.mean(historical_lead_time), 0))

    # Convert historical data to DataFrame
    simulation_results = pd.DataFrame(historical_data)

    # Plot simulation results if required
    print(plot_simulation)
    if plot_simulation:
        plot_inventory_simulation(simulation_results, time_series)

    return simulation_results


In [9]:
def plot_inventory_simulation(simulation_results: pd.DataFrame, demand_series: pd.Series) -> None:
    """
    Placeholder for the inventory simulation plotting function.

    Parameters:
    - simulation_results (pd.DataFrame): DataFrame containing the simulation results.
    - demand_series (pd.Series): Series of demand data.
    """

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=simulation_results['Date'], y=simulation_results['Stock'], mode='lines', name='Stock Level', line=dict(color='#1f77b4')))
    fig.add_trace(go.Scatter(x=simulation_results['Date'], y=simulation_results['ReorderPoint'], mode='lines', name='Reorder Point', line=dict(dash='dash', color='#2ca02c')))
    fig.add_trace(go.Scatter(x=simulation_results['Date'], y=simulation_results['SafetyStock'], mode='lines', name='Safety Stock', line=dict(dash='dot', color='#ff7f0e') ))
    fig.add_trace(go.Bar(x=simulation_results['Date'], y=simulation_results['Orders'], name='Ordered Quantity', marker=dict(color='#17becf')))
    # Demand
    fig.add_trace(go.Scatter(
        x=simulation_results['Date'],
        y=time_series,
        mode='lines',
        name='Demand',
        line=dict(color='#9467bd')
    ))

    # Identifying and marking zero stock points
    zero_stock_dates = simulation_results['Date'][simulation_results['Stock'] == 0]
    zero_stock_values = simulation_results['Stock'][simulation_results['Stock'] == 0]

    # Trace for zero stock weeks
    fig.add_trace(go.Scatter(
        x=zero_stock_dates,
        y=zero_stock_values,
        mode='markers',
        name='Zero Stock',
        marker=dict(color='red', size=12, symbol='x')
    ))


    # Count the number of orders made
    num_orders = len([order for order in simulation_results['Orders'] if order > 0])
    # Count the number of weeks with stock == 0
    num_stockout_weeks = (simulation_results['Stock'] == 0).sum()
    # Count the average stocked qty
    avg_stock = round(simulation_results['Stock'].mean(), 0)
    # Calculate the average quantity ordered
    avg_order_quantity = round(np.mean([order for order in simulation_results['Orders'] if order > 0]), 0)


    fig.update_layout(title=f'Inventory Simulation | #Orders: {num_orders} | #Stockouts {num_stockout_weeks} | AvgStockSize {avg_stock} | AvgOrderSize {avg_order_quantity}', xaxis_title='Date', yaxis_title='Stock Level')
    fig.show()

    pass

Change params and test how the simulation evolves

In [10]:
# How often we can send an order to the supplier, expressed in weeks
review_period = 1
# The service level we want to obtain, we set it to 90% this time
service_level_z = z_values_dict[90]
# Amount of weeks we use to calculate the avg and std of the demand
initial_weeks = 4
# Fixed lead time of the supplier, expressed in weeks
base_lead_time = 3

Test the difference of iventory by simulating the reorder process with and without a min. quantity to order

In [11]:
ignore_pending_orders = False
stochastic_lead_time = True
# minimum qty to order to supplier
min_order = 100

simulation_results = simulate_inventory(time_series, initial_weeks, review_period, base_lead_time, service_level_z, min_order, ignore_pending_orders, stochastic_lead_time, True)

True


In [12]:
ignore_pending_orders = False
# minimum qty to order to supplier
min_order = 0

simulation_results = simulate_inventory(time_series, initial_weeks, review_period, base_lead_time, service_level_z, min_order, ignore_pending_orders, stochastic_lead_time, True)

True
